In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [56]:
import numpy as np
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator, load_img
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import random 
import os
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import KFold
import tensorflow as tf




#print(os.listdir('./pet_data'))

In [3]:
# 기본경로+폴더명 작성  (폴더명을 os.listdir 로 가져와서 작업해도 됨)
base_dir = '/content/drive/MyDrive/Project II/pet_data'
train_dir = os.path.join(base_dir,'train')
train_cat_dir = os.path.join(train_dir,'cat')
train_dog_dir = os.path.join(train_dir,'dog')

# train_cat_dir = train["Id"].apply(lambda x:path +"/train/cat/"+ x + ".jpg")
# train_dog_dir = test["Id"].apply(lambda x:path +"/train/dog/"+ x + ".jpg")
print('end')

end


In [51]:
train=pd.read_csv('/content/drive/MyDrive/Project II/pet_data/n_dog.csv')

In [44]:
test=pd.read_csv('/content/drive/MyDrive/Project II/pet_data/test.csv')
test["file_path"] = test["Id"].apply(lambda x:"/content/drive/MyDrive/Project II/pet_data/original_test/"+ x + ".jpg")

In [43]:
test=pd.read_csv('/content/drive/MyDrive/Project II/pet_data/test.csv')

In [5]:
# 데이터 확인
train_cat_fnames = os.listdir(train_cat_dir)
train_dog_fnames = os.listdir(train_dog_dir)


['cat4319.jpg', 'cat4320.jpg', 'cat4321.jpg', 'cat4322.jpg', 'cat4323.jpg']
['dog3556.jpg', 'dog3557.jpg', 'dog3558.jpg', 'dog3559.jpg', 'dog3560.jpg']


In [39]:
path_dog="/content/drive/MyDrive/Project II/pet_data/train/dog/"

In [24]:
dog_list=[]
for i in train_dog_fnames:
    dog_list.append(path_dog+i)

In [26]:
image_size = 128
def preprocess(image_url):
    image_string = tf.io.read_file(image_url)
    image = tf.image.decode_jpeg(image_string, channels=3)
    image = tf.cast(image, tf.float32) / 255.0
    image = tf.image.resize(image, (image_size, image_size))
    return image
    

In [27]:

x_train=[]
for i in dog_list:
    x1=preprocess(i)
    x_train.append(x1)

In [45]:
test2=[]
for i in test['file_path']:
    x1=preprocess(i)
    test2.append(x1)
test2=np.array(test2)

In [46]:
test2

array([[[[0.18039216, 0.34509805, 0.4392157 ],
         [0.59607846, 0.7294118 , 0.8392157 ],
         [0.29803923, 0.34117648, 0.5176471 ],
         ...,
         [0.6313726 , 0.4745098 , 0.42745098],
         [0.34901962, 0.08235294, 0.03137255],
         [0.6745098 , 0.3529412 , 0.29803923]],

        [[0.36862746, 0.5294118 , 0.6392157 ],
         [0.20784314, 0.3137255 , 0.41960785],
         [0.48235294, 0.48235294, 0.5764706 ],
         ...,
         [0.80784315, 0.7294118 , 0.62352943],
         [0.63529414, 0.4627451 , 0.41960785],
         [0.54509807, 0.3254902 , 0.32156864]],

        [[0.07843138, 0.24705882, 0.38039216],
         [0.10980392, 0.19215687, 0.26666668],
         [0.8666667 , 0.78039217, 0.7254902 ],
         ...,
         [0.5686275 , 0.64705884, 0.41568628],
         [0.654902  , 0.6745098 , 0.65882355],
         [0.28235295, 0.26666668, 0.35686275]],

        ...,

        [[0.67058825, 0.7176471 , 0.6627451 ],
         [0.27450982, 0.27450982, 0.24313726]

In [52]:
x_train=np.array(x_train)
y_train=train['Pawpularity']
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x_train,y_train,test_size=0.2)

In [57]:
inputs=keras.Input(shape=(128,128,3))
x=inputs
x=keras.layers.Conv2D(filters=4,kernel_size=3,strides=2,padding='same',activation='relu')(x)
x=keras.layers.Conv2D(filters=8,kernel_size=3,strides=2,padding='same',activation='relu')(x)
x=keras.layers.Conv2D(filters=16,kernel_size=3,strides=2,padding='same',activation='relu')(x)
x=keras.layers.Conv2D(filters=32,kernel_size=3,strides=2,padding='same',activation='relu')(x)
x=keras.layers.Flatten()(x)
x=keras.layers.Dense(128, activation = "relu")(x)
x=keras.layers.Dropout(0.5)(x)
output = tf.keras.layers.Dense(1)(x)
model = tf.keras.Model(inputs=inputs, outputs=output)

In [58]:
early_stop = tf.keras.callbacks.EarlyStopping(
    patience=5
    )
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    factor=0.1,
    patience=2, 
    min_lr=1e-9
    )
callbacks = [early_stop,reduce_lr]

In [59]:
model.compile(loss='mse', optimizer='Adam', metrics=[tf.keras.metrics.RootMeanSquaredError(name="rmse"), "mae", "mape"])

In [60]:
history = model.fit(x_train,y_train, epochs=25,batch_size=32,validation_data = (x_test,y_test),callbacks=callbacks)

Epoch 1/25
114/114 [==============================] - 8s 61ms/step - loss: 792.0937 - rmse: 28.1442 - mae: 21.5247 - mape: 98.1224 - val_loss: 558.5179 - val_rmse: 23.6330 - val_mae: 18.5684 - val_mape: 95.5788
Epoch 2/25
114/114 [==============================] - 7s 58ms/step - loss: 592.9718 - rmse: 24.3510 - mae: 18.6905 - mape: 96.1705 - val_loss: 560.2816 - val_rmse: 23.6703 - val_mae: 17.6094 - val_mape: 82.4582
Epoch 3/25
114/114 [==============================] - 7s 59ms/step - loss: 582.8656 - rmse: 24.1426 - mae: 18.5803 - mape: 96.3757 - val_loss: 542.5895 - val_rmse: 23.2936 - val_mae: 18.2853 - val_mape: 94.0932
Epoch 4/25
114/114 [==============================] - 7s 60ms/step - loss: 579.4807 - rmse: 24.0724 - mae: 18.4387 - mape: 97.1580 - val_loss: 546.1342 - val_rmse: 23.3695 - val_mae: 17.8637 - val_mape: 87.5565
Epoch 5/25
114/114 [==============================] - 7s 62ms/step - loss: 574.5064 - rmse: 23.9689 - mae: 18.4242 - mape: 96.9176 - val_loss: 583.1448 - va

In [61]:
cnn_pred=model.predict(test2)
cnn_pred

array([[41.3394  ],
       [42.160107],
       [41.93188 ],
       [41.137524],
       [41.74199 ],
       [42.123005],
       [40.925694],
       [42.240955]], dtype=float32)

- 모델 구성